<a href="https://colab.research.google.com/github/Akash-Git20/bitcoin_price_prediction/blob/main/Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import os
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [2]:
tz = pytz.timezone("America/New_York")
start = tz.localize(dt(2014,1,1))
end = tz.localize(dt.today())
tickers = "BTC-USD"
df = yf.download(tickers,start, end, auto_adjust=True)

[*********************100%%**********************]  1 of 1 completed


In [3]:
df.index = pd.to_datetime(df.index)
df["Tomorrow"] = df["Close"].shift(-1)
df["Target"] = (df["Tomorrow"] > df["Close"]).astype(int)

In [4]:
def predict(train, test, predictors, model):
  model.fit(train[predictors], train["Target"])
  preds = model.predict(test[predictors])
  preds = pd.Series(preds, index=test.index, name="Predictions")
  combined = pd.concat([test["Target"], preds], axis=1)
  return combined

In [5]:
def get_precision_score(arr):
  return precision_score(arr["Target"], arr["Predictions"])

In [6]:
train = df.iloc[:-200]
test = df.iloc[-200:]

In [7]:
from sklearn.linear_model import LogisticRegression
logisticModel = LogisticRegression(random_state=0)

In [8]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(n_estimators = 50, min_samples_split=50, random_state=1)

In [19]:
from xgboost import XGBClassifier
modelXGB = XGBClassifier(random_state = 1, learning_rate = .1, n_estimators=200)

In [10]:
predictors = ["Close", "High", "Low", "Open", "Volume"]

In [9]:
def backtest(data, model, predictors, start = 1050, step = 150):
  all_predictions = []
  for i in range(start, data.shape[0], step):
    train = data.iloc[0:i].copy()
    test = data.iloc[i:(i + step)].copy()
    predictions = predict(train, test, predictors, model)
    all_predictions.append(predictions)
  return pd.concat(all_predictions)

In [17]:
predictions_RF = backtest(train, model, predictors)

In [18]:
get_precision_score(predictions_RF)

0.49754500818330605

In [15]:
predictions_LR = backtest(train, logisticModel, predictors)

In [16]:
get_precision_score(predictions_LR)

0.5202863961813843

In [20]:
predictions_XGB = backtest(train, modelXGB, predictors)

In [21]:
get_precision_score(predictions_XGB)

0.5143769968051118